In [24]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
matrix_for_extraction = 'GP_TotTime'

# List of TAZ rows to extract
taz_rows_to_extract = [ 832, # Bountiful
                       1629, # Murray
                       1984, # South Jordan
                       3167, # Spanish Fork
                       2845, # MAG Office
                       2513] # Saratoga

In [26]:
# Define the input OMX file and output CSV file
omx_file = "data/input_matrix.omx"
output_csv = f"intermediate/TAZ-I-J-{matrix_for_extraction}.csv"

In [27]:
matrix_rows_to_extract = [i - 1 for i in taz_rows_to_extract]  # Convert to zero-based index

# Open the OMX file
with omx.open_file(omx_file, 'r') as omx_data:
    # Read the entire matrix_for_extraction matrix
    _matrix = omx_data[matrix_for_extraction][:]

    # Extract I-J-Value data
    extracted_data = []
    for i in matrix_rows_to_extract:
        for j in range(_matrix.shape[1]):  # Iterate over all columns
            extracted_data.append([i + 1, j + 1, _matrix[i, j]])  # Convert back to 1-based for output

# Convert to DataFrame
extracted_data_df = pd.DataFrame(extracted_data, columns=['I', 'J', 'Value'])

display(extracted_data_df)

# Save to CSV
extracted_data_df.to_csv(output_csv, index=False)

print(f"Extracted data saved to {output_csv}")

,I,J,Value
0,832,1,77.70
1,832,2,77.14
2,832,3,72.45
3,832,4,73.51
4,832,5,76.02
...,...,...,...
10882,1984,3625,160.88
10883,1984,3626,100.00
10884,1984,3627,141.18
10885,1984,3628,76.42


Extracted data saved to intermediate/TAZ-I-J-GP_TotTime.csv


In [28]:
travel_shed_ato_classes_df = pd.read_csv("data/NEW And OLD ATO Method (for travelsheds).csv")
travel_shed_ato_classes_df

,Method,TTStart_Minutes,TTEnd_Minutes,MAPCLASS,JOBFRACT
0,New,0.0000,10.00,11,1
1,New,10.0010,13.50,10,0.90 - 0.99
2,New,13.5010,17.00,9,0.80 - 0.89
3,New,17.0010,20.50,8,0.70 - 0.79
4,New,20.5010,24.00,7,0.60 - 0.69
5,New,24.0010,27.50,6,0.50 - 0.59
6,New,27.5010,31.00,5,0.40 - 0.49
7,New,31.0010,34.50,4,0.30 - 0.39
8,New,34.5010,38.00,3,0.20 - 0.29
9,New,38.0010,41.50,2,0.10 - 0.19


In [29]:
# Performing the join based on the Value being between TTStart_Minutes and TTEnd_Minutes
merged_df = extracted_data_df.merge(
    travel_shed_ato_classes_df,
    how="cross"
).query("TTStart_Minutes <= Value < TTEnd_Minutes")

# Display the merged DataFrame
display(merged_df)


,I,J,Value,Method,TTStart_Minutes,TTEnd_Minutes,MAPCLASS,JOBFRACT
65,832,3,72.45,Old,39.251,75.00,1,0.00 - 0.09
87,832,4,73.51,Old,39.251,75.00,1,0.00 - 0.09
131,832,6,72.12,Old,39.251,75.00,1,0.00 - 0.09
153,832,7,72.36,Old,39.251,75.00,1,0.00 - 0.09
175,832,8,72.90,Old,39.251,75.00,1,0.00 - 0.09
...,...,...,...,...,...,...,...,...
239182,1984,3614,36.31,Old,29.251,39.25,2,0.10 - 0.19
239205,1984,3615,48.10,Old,39.251,75.00,1,0.00 - 0.09
239271,1984,3618,62.65,Old,39.251,75.00,1,0.00 - 0.09
239293,1984,3619,60.81,Old,39.251,75.00,1,0.00 - 0.09


In [35]:
# Load the TAZ GeoJSON file
taz_geo = gpd.read_file("data/WFv910_TAZ.geojson")
taz_df = pd.read_csv("data/WFv910_TAZ.csv")
taz_geo_merged = pd.merge(taz_geo, taz_df, on="TAZID")
taz_geo_merged = taz_geo_merged[taz_geo_merged['REMM']==1]
taz_geo = taz_geo_merged
#display(taz_geo_merged)

# Merge TAZ data with the extracted dataset using 'TAZID'
merged_geo = taz_geo.merge(merged_df, left_on="TAZID", right_on="J")

# Get unique 'I' values
unique_I_values = merged_geo["I"].unique()

# Set up Chrome WebDriver for capturing PNGs
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1200x800")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Custom RGB color palette for MAPCLASS (converted to HEX)
mapclass_colors = {
    11: "#ACFA70",  # (172,250,112)
    10: "#70EA80",  # (112,234,128)
    9:  "#23D890",  # (35,216,144)
    8:  "#00C49E",  # (0,196,158)
    7:  "#00ADA4",  # (0,173,164)
    6:  "#0097A3",  # (0,151,163)
    5:  "#00829D",  # (0,130,157)
    4:  "#006D95",  # (0,109,149)
    3:  "#005886",  # (0,88,134)
    2:  "#204370",  # (32,67,112)
    1:  "#292F56",  # (41,47,86)
}

# Function to create a Folium map with custom colors and matching polygon borders
def create_map(geo_data, title, center, zoom_level):
    """Creates a Folium map with a custom color scheme and matching polygon border colors."""
    map_obj = folium.Map(location=center, zoom_start=zoom_level)

    # Style function to apply custom colors and matching border color
    def style_function(feature):
        mapclass = feature["properties"]["MAPCLASS"]
        fill_color = mapclass_colors.get(mapclass, "#FFFFFF")  # Default to white if not found
        return {
            "fillColor": fill_color,
            "color": fill_color,  # **MATCH BORDER TO FILL COLOR**
            "weight": 0.5,  # Slight weight to avoid slivers
            "fillOpacity": 0.85
        }

    folium.GeoJson(
        geo_data,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=["MAPCLASS"])
    ).add_to(map_obj)

    return map_obj

# Folder to store PNGs
output_folder = "output_maps"
os.makedirs(output_folder, exist_ok=True)

# Loop through each unique I value to create separate maps
for i_val in unique_I_values:
    subset_geo = merged_geo[merged_geo["I"] == i_val]

    # Separate datasets for New and Old methods
    new_geo = subset_geo[subset_geo["Method"] == "New"]
    old_geo = subset_geo[subset_geo["Method"] == "Old"]

    # Define a consistent zoom extent based on the full dataset for `I`
    center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
    zoom_level = 10  # Adjust if necessary

    # Create maps with the same zoom extent
    new_map = create_map(new_geo, f"New Method - I={i_val}", center, zoom_level)
    old_map = create_map(old_geo, f"Old Method - I={i_val}", center, zoom_level)

    # Save maps as temporary HTML files
    new_map_html = f"output_html/map_temp_new_I_{i_val}.html"
    old_map_html = f"output_html/map_temp_old_I_{i_val}.html"
    
    new_map.save(new_map_html)
    old_map.save(old_map_html)

    # Convert HTML to PNG
    def save_as_png(html_file, output_file):
        """Renders the HTML map and saves it as a PNG image."""
        file_url = f"file://{os.path.abspath(html_file)}"
        driver.get(file_url)
        driver.save_screenshot(output_file)
        print(f"Saved: {output_file}")

    # File names with I at the beginning
    new_png = os.path.join(output_folder, f"{i_val}_New_Method.png")
    old_png = os.path.join(output_folder, f"{i_val}_Old_Method.png")

    save_as_png(new_map_html, new_png)
    save_as_png(old_map_html, old_png)

    # Clean up temporary HTML files
    os.remove(new_map_html)
    os.remove(old_map_html)

# Close the WebDriver
driver.quit()

print(f"All PNG maps saved in '{output_folder}' folder.")


C:\Users\bhereth\AppData\Local\Temp\ipykernel_15124\273152181.py:75: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]


Saved: output_maps\832_New_Method.png
Saved: output_maps\832_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_15124\273152181.py:75: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]


Saved: output_maps\1629_New_Method.png
Saved: output_maps\1629_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_15124\273152181.py:75: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]


Saved: output_maps\1984_New_Method.png
Saved: output_maps\1984_Old_Method.png
All PNG maps saved in 'output_maps' folder.
